# Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
images_path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA"

Mounted at /content/drive


# Resize

In [ ]:
from PIL import Image
import os

# Target size for every image
target_size = (500, 500)
os.makedirs("data/resized500", exist_ok=True)

# Iterating through class_0 images to resize
for image_file in os.listdir("{}/class_0".format(images_path)):
    image_path = os.path.join("{}/class_0".format(images_path), image_file)
    img = Image.open(image_path)
    img_resized = img.resize(target_size)

    #kaydet
    output_path = os.path.join("data/resized500", image_file)
    img_resized.save(output_path)

# Iterating through class_1 images to resize
for image_file in os.listdir("{}/class_1".format(images_path)):
    image_path = os.path.join("{}/class_1".format(images_path), image_file)
    img = Image.open(image_path)
    img_resized = img.resize(target_size)

    #kaydet
    output_path = os.path.join("data/resized500", image_file)
    img_resized.save(output_path)

print("Image resizing complete.")

Image resizing complete.


# Splitting Data Set Into Training and Test

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
images_path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA"

# Path to store training and test datasets
destination_folder = "/content/drive/MyDrive/Traffic Sign Detection/cnn1/training and test"

# Class names
classes = ["class_0", "class_1"]

# Create train and test folders
train_folder = os.path.join(destination_folder, "train")
test_folder = os.path.join(destination_folder, "test")

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [ ]:
# Copy images from original dataset to train and test folders
for class_name in classes:
    class_folder = os.path.join(images_path, class_name)

    # Create class folders in train and test folders
    train_class_folder = os.path.join(train_folder, class_name)
    test_class_folder = os.path.join(test_folder, class_name)

    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)


    # Split images into train and test sets
    image_list = os.listdir(class_folder)
    train_images, test_images = train_test_split(image_list, test_size=0.2, random_state=42)

    # Copy images to train folder
    for img in train_images:
        source_path = os.path.join(class_folder, img)
        destination_path = os.path.join(train_class_folder, img)
        shutil.copy(source_path, destination_path)

    # Copy images to test folder
    for img in test_images:
        source_path = os.path.join(class_folder, img)
        destination_path = os.path.join(test_class_folder, img)
        shutil.copy(source_path, destination_path)

print("Dataset split into train and test folders.")

Dataset split into train and test folders.


# CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

In [ ]:
# Paths to store splitted datasets
train_data_dir = "/content/drive/MyDrive/Traffic Sign Detection/cnn1/training and test/train"
test_data_dir = "/content/drive/MyDrive/Traffic Sign Detection/cnn1/training and test/test"

batch_size = 32

# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values to the range [0, 1]
    shear_range=0.2,     # Shear transformations
    zoom_range=0.2,      # Zoom transformations
    horizontal_flip=True  # Horizontal flips
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create train generators with established batch size
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(500, 500),
    batch_size=batch_size,
    class_mode="binary"  # Assumes two classes
)

# Create validation generators with established batch size
validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(500, 500),
    batch_size=batch_size,
    class_mode="binary")

Found 870 images belonging to 2 classes.
Found 272 images belonging to 2 classes.


In [ ]:
# Creating model and layers to train
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(500, 500, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))  # Binary classification, so use sigmoid activation

# Compiling the model and setting parameters
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Setting Checkpoint to save the best model
checkpoint = ModelCheckpoint("best_tsmodel1.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1)

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks = [checkpoint]
)

# Saving the model
model.save("/content/drive/MyDrive/Traffic Sign Detection/cnn1/models/best_tsmodel1.h5")

Epoch 1/20
27/27 [==============================] - ETA: 0s - loss: 0.7177 - accuracy: 0.7053
Epoch 1: val_accuracy improved from -inf to 0.64844, saving model to best_tsmodel1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


27/27 [==============================] - 110s 4s/step - loss: 0.7177 - accuracy: 0.7053 - val_loss: 0.5787 - val_accuracy: 0.6484
Epoch 2/20
27/27 [==============================] - ETA: 0s - loss: 0.4717 - accuracy: 0.7721
Epoch 2: val_accuracy improved from 0.64844 to 0.76953, saving model to best_tsmodel1.h5
27/27 [==============================] - 87s 3s/step - loss: 0.4717 - accuracy: 0.7721 - val_loss: 0.4734 - val_accuracy: 0.7695
Epoch 3/20
27/27 [==============================] - ETA: 0s - loss: 0.4224 - accuracy: 0.8210
Epoch 3: val_accuracy improved from 0.76953 to 0.79297, saving model to best_tsmodel1.h5
27/27 [==============================] - 97s 4s/step - loss: 0.4224 - accuracy: 0.8210 - val_loss: 0.4409 - val_accuracy: 0.7930
Epoch 4/20
27/27 [==============================] - ETA: 0s - loss: 0.3958 - accuracy: 0.8341
Epoch 4: val_accuracy improved from 0.79297 to 0.83594, saving model to best_tsmodel1.h5
27/27 [==============================] - 99s 4s/step - loss: 0.

In [ ]:
# Loading the model and seeing results for the whole dataset
model = tf.keras.models.load_model("/content/drive/MyDrive/Traffic Sign Detection/cnn1/models/best_tsmodel1.h5")

confusion = [[0, 0], [0, 0]]

# CLASS_0
counter = [0, 0]
path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA/class_0"
for filename in os.listdir(path):
    # Loading and preprocessing the test image
    img_path = os.path.join(path, filename)
    img = load_img(img_path, target_size=(500, 500))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]

    # Making a prediction
    prediction = model.predict(img_array)

    # Printing the predicted class (0 or 1)
    print(f"Image: {filename}, Prediction: {int(round(prediction[0][0]))}")
    if int(round(prediction[0][0])) == 0:
      counter[0] += 1
    elif int(round(prediction[0][0])) == 1:
      counter[1] += 1

confusion[1][1], confusion[0][1] = counter[0], counter[1]

# CLASS_1
counter = [0, 0]
path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA/class_1"
for filename in os.listdir(path):
    # Loading and preprocessing the test image
    img_path = os.path.join(path, filename)
    img = load_img(img_path, target_size=(500, 500))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]

    # Making a prediction
    prediction = model.predict(img_array)

    # Printing the predicted class (0 or 1)
    print(f"Image: {filename}, Prediction: {int(round(prediction[0][0]))}")
    if int(round(prediction[0][0])) == 0:
      counter[0] += 1
    elif int(round(prediction[0][0])) == 1:
      counter[1] += 1
print(counter)


confusion[0][0], confusion[1][0] = counter[1], counter[0]

1/1 [==============================] - 0s 89ms/step
Image: road12.png, Prediction: 0
1/1 [==============================] - 0s 21ms/step
Image: road11.png, Prediction: 0
1/1 [==============================] - 0s 54ms/step
Image: road0.png, Prediction: 0
1/1 [==============================] - 0s 21ms/step
Image: road10.png, Prediction: 0
1/1 [==============================] - 0s 21ms/step
Image: road1.png, Prediction: 0
1/1 [==============================] - 0s 20ms/step
Image: road15.png, Prediction: 0
1/1 [==============================] - 0s 20ms/step
Image: road16.png, Prediction: 1
1/1 [==============================] - 0s 20ms/step
Image: road17.png, Prediction: 0
1/1 [==============================] - 0s 23ms/step
Image: road19.png, Prediction: 0
1/1 [==============================] - 0s 20ms/step
Image: road18.png, Prediction: 0
1/1 [==============================] - 0s 34ms/step
Image: road14.png, Prediction: 1
1/1 [==============================] - 0s 20ms/step
Image: road13.p

In [ ]:
# Printing the confusion matrix
for i in confusion:
  print(i)

[749, 29]
[40, 205]
